In [1]:
import ollama
import time
import os
import json
import numpy as np
from numpy.linalg import norm
import PyPDF2
from docx import Document
from sklearn.decomposition import PCA  # Menambahkan PCA untuk reduksi dimensi jika perlu

# Membaca file TXT
def read_txt(filename):
    with open(filename, "r", encoding="utf-8-sig") as f:
        return f.read()

# Membaca file PDF
def read_pdf(filename):
    with open(filename, "rb") as f:
        pdf_reader = PyPDF2.PdfReader(f)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text

# Membaca file DOCX
def read_docx(filename):
    doc = Document(filename)
    text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
    return text

# Mendapatkan paragraf dari semua file
def parse_file(filename):
    if filename.endswith(".txt"):
        content = read_txt(filename)
    elif filename.endswith(".pdf"):
        content = read_pdf(filename)
    elif filename.endswith(".docx"):
        content = read_docx(filename)
    else:
        raise ValueError(f"Unsupported file type: {filename}")
    
    paragraphs = []
    buffer = []
    for line in content.splitlines():
        line = line.strip()
        if line:
            buffer.append(line)
        elif len(buffer):
            paragraphs.append(" ".join(buffer))
            buffer = []
    if len(buffer):
        paragraphs.append(" ".join(buffer))
    return paragraphs

# Simpan embedding ke file
def save_embeddings(filename, embeddings):
    if not os.path.exists("embeddings"):
        os.makedirs("embeddings")
    with open(f"embeddings/{filename}.json", "w") as f:
        json.dump(embeddings, f)

# Memuat embedding dari file
def load_embeddings(filename):
    if not os.path.exists(f"embeddings/{filename}.json"):
        return False
    with open(f"embeddings/{filename}.json", "r") as f:
        return json.load(f)

# Mendapatkan embedding
def get_embeddings(filename, modelname, chunks):
    if (embeddings := load_embeddings(filename)) is not False:
        return embeddings
    embeddings = [
        ollama.embeddings(model=modelname, prompt=chunk)["embedding"]
        for chunk in chunks
    ]
    
    # Reduksi dimensi jika diperlukan (misalnya PCA)
    reduced_embeddings = reduce_embeddings_dimension(embeddings)
    
    save_embeddings(filename, reduced_embeddings)
    return reduced_embeddings

# Reduksi Dimensi dengan PCA (jika diperlukan)
def reduce_embeddings_dimension(embeddings):
    embeddings_array = np.array(embeddings)
    
    # Mengecek dimensi pertama
    if embeddings_array.shape[1] > 768:  # Jika dimensi lebih besar dari 768 (misalnya 4096)
        pca = PCA(n_components=768)  # Mengurangi dimensi ke 768
        embeddings_array = pca.fit_transform(embeddings_array)  # Reduksi dimensi
        print(f"Reduksi dimensi menjadi: {embeddings_array.shape}")
    
    return embeddings_array.tolist()  # Mengembalikan sebagai list jika diperlukan

# Cosine similarity untuk menemukan kemiripan
def find_most_similar(needle, haystack):
    needle_norm = norm(needle)
    similarity_scores = [
        np.dot(needle, item) / (needle_norm * norm(item)) for item in haystack
    ]
    return sorted(zip(similarity_scores, range(len(haystack))), reverse=True)

In [3]:
# Fungsi utama
def main():
    SYSTEM_PROMPT = """You are an assistant that answers questions only in Bahasa Indonesia. 
    Your answers must be based solely on the provided context extracted from the documents. 
    If the answer cannot be determined from the context, respond with "Maaf, saya tidak tahu." 
    Do not include any information outside of the given context, and strictly reply in Bahasa Indonesia.

    Context:
    """

    data_folder = "data/IT"
    all_paragraphs = []
    filenames = []
    
    # Iterasi semua file dalam folder data
    for file in os.listdir(data_folder):
        file_path = os.path.join(data_folder, file)
        if file.lower().endswith((".txt", ".pdf", ".docx")):
            paragraphs = parse_file(file_path)
            all_paragraphs.extend(paragraphs)
            filenames.append(file)
    
    # Ambil nama folder terakhir (misalnya "IT" atau "Fikri")
    folder_name = os.path.basename(data_folder)

    # Buat nama file embeddings berdasarkan nama folder
    embeddings_filename = f"data_embeddings_{folder_name}"
    # Buat embedding
    embeddings = get_embeddings(embeddings_filename, "nomic-embed-text", all_paragraphs)

    while True:
        prompt = input("Silakan tanya bosku? (ketik 'exit' untuk keluar) -> ")
        
        if prompt.lower() == "exit":
            print("Exiting the assistant. Goodbye!")
            break

        prompt_embedding = ollama.embeddings(model="nomic-embed-text", prompt=prompt)["embedding"]
        
        # Reduksi dimensi jika perlu pada prompt embedding (jika diperlukan)
        prompt_embedding = reduce_embeddings_dimension([prompt_embedding])[0]  # Reduksi menjadi 768 dimensi

        most_similar_chunks = find_most_similar(prompt_embedding, embeddings)[:10]

        response = ollama.chat(
            model="llama3",
            messages=[
                {
                    "role": "system",
                    "content": SYSTEM_PROMPT
                    + "\n".join(all_paragraphs[item[1]] for item in most_similar_chunks),
                },
                {"role": "user", "content": prompt},
            ],
        )
        
        print("\n\n")
        print(response["message"]["content"])


if __name__ == "__main__":
    main()

Silakan tanya bosku? (ketik 'exit' untuk keluar) ->  apa saja point dari dokumen dokumen ini?





Maaf, saya tidak tahu. Tidak ada informasi yang diketahui tentang poin dari dokumen-dokumen tersebut.


Silakan tanya bosku? (ketik 'exit' untuk keluar) ->  lalu apa yg anda ketahui dari konteks?





Dari konteks yang diberikan, saya mengetahui bahwa:

1. Mereka membahas tentang cara mencetak dokumen dari laptop.
2. Mereka juga membahas tentang penggunaan ekstensi browser untuk VPN (Virtual Private Network).
3. Ada beberapa contoh jawaban dengan empati untuk berbagai situasi, seperti laptop yang mati, koneksi internet lambat, lupa password email, dan masalah dengan printer.

Saya juga mengetahui bahwa tips untuk keamanan VPN adalah menggunakan layanan VPN yang tepercaya dan hindari layanan VPN gratis.


Silakan tanya bosku? (ketik 'exit' untuk keluar) ->  coba jabarkan keseluruhan informasi terkait penggunaan VPN





Bagaimana cara mengaktifkan VPN? Mengaktifkan VPN (Virtual Private Network) tergantung pada perangkat dan layanan VPN yang Anda gunakan. Berikut adalah langkah-langkah umum untuk berbagai perangkat dan situasi:

1. Menggunakan Aplikasi VPN (Penyedia Layanan VPN)
   Jika Anda memiliki langganan layanan VPN (seperti NordVPN, ExpressVPN, ProtonVPN, dll.):
   Unduh aplikasi resmi dari penyedia VPN di perangkat Anda (PC, Mac, Android, atau iOS).
   Masuk ke Akun Anda: Buka aplikasi dan login dengan akun yang Anda buat.
   Pilih Server: Pilih lokasi server yang ingin Anda gunakan (misalnya, negara tertentu).
   Aktifkan VPN: Klik tombol Connect atau On untuk mengaktifkan VPN.
   Periksa Status: Pastikan indikator VPN aktif (biasanya berupa ikon kunci atau tanda VPN di bilah notifikasi perangkat Anda).

2. Menggunakan Fitur Bawaan di Perangkat
   a. Di Android:
      Tambahkan Profil VPN: Klik Add VPN Profile.
      Masukkan informasi server VPN (alamat server, username, password).
      H

Silakan tanya bosku? (ketik 'exit' untuk keluar) ->  siapa fikri rama?





Maaf, saya tidak tahu.


Silakan tanya bosku? (ketik 'exit' untuk keluar) ->  exit


Exiting the assistant. Goodbye!
